Петухова Ксения

Загружаем и импортируем необходимые модули

In [4]:
!pip install h5py

In [5]:
!pip install typing-extensions

In [2]:
!pip install wheel

In [6]:
!pip install -q -U watermark

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 2.1.3 which is incompatible.


In [7]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 46.6 MB/s 
     |████████████████████████████████| 6.5 MB 23.0 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 895 kB 36.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 5.5.0

numpy       : 1.21.5
pandas      : 1.3.5
torch       : 1.10.0+cu111
transformers: 4.17.0



In [10]:
!pip install datasets

     |████████████████████████████████| 325 kB 5.0 MB/s 
     |████████████████████████████████| 134 kB 6.5 MB/s 
     |████████████████████████████████| 212 kB 36.5 MB/s 
     |████████████████████████████████| 1.1 MB 9.5 MB/s 
     |████████████████████████████████| 127 kB 27.8 MB/s 
     |████████████████████████████████| 94 kB 1.3 MB/s 
     |████████████████████████████████| 271 kB 34.2 MB/s 
     |████████████████████████████████| 144 kB 28.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install wandb

In [9]:
import transformers
from transformers import BertModel, BertTokenizer, PreTrainedTokenizerFast, AdamW, get_linear_schedule_with_warmup, Trainer, TrainingArguments 
import torch.nn.functional as F
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support, accuracy_score
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import wandb
from transformers import BertForSequenceClassification, TextClassificationPipeline
wandb.init(mode="disabled")

In [11]:
RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Готовим данные

In [12]:
imdb_dataset = load_dataset('imdb')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [14]:
imdb_dataset = imdb_dataset.shuffle(seed=42)

In [15]:
imdb_dataset["train"]["text"][:10]

['There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...',
 'This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stub your toe on the moon" It reminds m

In [16]:
imdb_dataset["train"]["label"][:10]

[1, 1, 0, 1, 0, 1, 1, 0, 0, 1]

In [17]:
train_texts = imdb_dataset["train"]["text"][:2000]
train_labels = imdb_dataset["train"]["label"][:2000]
test_texts = imdb_dataset["test"]["text"][:1000]
test_labels = imdb_dataset["test"]["label"][:1000]

In [18]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

Загружаем маленькую бертовскую модель

In [19]:
PRE_TRAINED_MODEL_NAME = 'prajjwal1/bert-tiny'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME, model_max_length=512)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/285 [00:00<?, ?B/s]

Токенизируем наши датасеты с помощью предобученного токенайзера

In [20]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

Создадим класс Dataset, который на каждой итерации будет возвращать токенизированный отзыв и класс

In [21]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

Функция для подсчета метрик

In [22]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Зададим параметры обучения с помощью TrainingArguments:

In [23]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

### Задание 1

Опишем наш классификатор на основе BertModel

In [24]:
class SentimentClassifier(nn.Module):

    def __init__(self, n_classes):
        super().__init__()
        self.n_classes = n_classes
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, token_type_ids, labels):
        _, pooled_output = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask,
        return_dict=False)
        output = self.drop(pooled_output)
        logits = self.out(output)
        loss = self.loss(logits.view(-1, self.n_classes), labels.view(-1))
        return loss, logits

Инициализируем модель

In [25]:
model = SentimentClassifier(2)
model = model.to(device)

Downloading:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Запустим файнтюнинг модели с помощью модуля Trainer

In [28]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1600
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 200


Step,Training Loss
10,0.723300
20,0.735100
30,0.716800
40,0.693300
50,0.694300
60,0.715600
70,0.698100
80,0.697900
90,0.698100
100,0.714000




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=200, training_loss=0.6967517638206482, metrics={'train_runtime': 18.1701, 'train_samples_per_second': 176.114, 'train_steps_per_second': 11.007, 'total_flos': 0.0, 'train_loss': 0.6967517638206482, 'epoch': 2.0})

In [29]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 400
  Batch size = 64


{'epoch': 2.0,
 'eval_accuracy': 0.6675,
 'eval_f1': 0.6840855106888362,
 'eval_loss': 0.6440865397453308,
 'eval_precision': 0.6792452830188679,
 'eval_recall': 0.6889952153110048,
 'eval_runtime': 0.7405,
 'eval_samples_per_second': 540.149,
 'eval_steps_per_second': 9.453}

Оценим модель на тестовых данных

In [30]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64


{'epoch': 2.0,
 'test_accuracy': 0.626,
 'test_f1': 0.622983870967742,
 'test_loss': 0.6535375714302063,
 'test_precision': 0.6130952380952381,
 'test_recall': 0.6331967213114754,
 'test_runtime': 1.8264,
 'test_samples_per_second': 547.511,
 'test_steps_per_second': 8.76}

### Задание 2

Изменим модель, чтобы помимо выхода с пуллер-слоя использовался эмбеддинг cls-токена с последнего слоя

In [31]:
class SentimentClassifier_cls(nn.Module):

    def __init__(self, n_classes):
        super().__init__()
        self.n_classes = n_classes
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size*2, n_classes)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, token_type_ids, labels):
        last_hidden_state, pooled_output, hidden_states = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask,
        return_dict=False,
        output_hidden_states=True)
        output = self.drop(torch.cat((pooled_output, last_hidden_state[:, 0, :]), dim=1))
        logits = self.out(output)
        loss = self.loss(logits.view(-1, self.n_classes), labels.view(-1))
        return loss, logits

Инициализируем модель

In [32]:
model_cls = SentimentClassifier_cls(2)
model_cls = model_cls.to(device)

loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/1ee037c9e1a220d5c814779ffe697080d1e6f5b16

Запустим файнтюнинг модели с помощью модуля Trainer

In [33]:
trainer = Trainer(
    model=model_cls,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1600
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 200


Step,Training Loss
10,0.713900
20,0.726000
30,0.787100
40,0.701000
50,0.697100
60,0.705000
70,0.741600
80,0.692700
90,0.718800
100,0.747200




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=200, training_loss=0.7191409492492675, metrics={'train_runtime': 16.0541, 'train_samples_per_second': 199.326, 'train_steps_per_second': 12.458, 'total_flos': 0.0, 'train_loss': 0.7191409492492675, 'epoch': 2.0})

In [34]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 400
  Batch size = 64


{'epoch': 2.0,
 'eval_accuracy': 0.6775,
 'eval_f1': 0.7237687366167023,
 'eval_loss': 0.6303771138191223,
 'eval_precision': 0.6550387596899225,
 'eval_recall': 0.8086124401913876,
 'eval_runtime': 0.7788,
 'eval_samples_per_second': 513.579,
 'eval_steps_per_second': 8.988}

Оценим модель на тестовых данных

In [35]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64


{'epoch': 2.0,
 'test_accuracy': 0.588,
 'test_f1': 0.6192236598890943,
 'test_loss': 0.6712096929550171,
 'test_precision': 0.563973063973064,
 'test_recall': 0.6864754098360656,
 'test_runtime': 1.6514,
 'test_samples_per_second': 605.559,
 'test_steps_per_second': 9.689}

### Задание 3

Применим к данным готовую модель для классификации последовательности

In [36]:
model_pretrained = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME)

loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/1ee037c9e1a220d5c814779ffe697080d1e6f5b16

Запустим файнтюнинг модели с помощью модуля Trainer

In [37]:
trainer = Trainer(
    model=model_pretrained,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1600
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 200


Step,Training Loss
10,0.700800
20,0.689900
30,0.709700
40,0.715700
50,0.729100
60,0.683300
70,0.716500
80,0.700500
90,0.696600
100,0.693300




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=200, training_loss=0.6929146671295165, metrics={'train_runtime': 16.2982, 'train_samples_per_second': 196.341, 'train_steps_per_second': 12.271, 'total_flos': 4065558528000.0, 'train_loss': 0.6929146671295165, 'epoch': 2.0})

In [38]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 400
  Batch size = 64


{'epoch': 2.0,
 'eval_accuracy': 0.7125,
 'eval_f1': 0.7368421052631579,
 'eval_loss': 0.6661474704742432,
 'eval_precision': 0.706140350877193,
 'eval_recall': 0.7703349282296651,
 'eval_runtime': 0.5819,
 'eval_samples_per_second': 687.445,
 'eval_steps_per_second': 12.03}

Оценим модель на тестовых данных

In [39]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64


{'epoch': 2.0,
 'test_accuracy': 0.682,
 'test_f1': 0.6741803278688525,
 'test_loss': 0.6718357801437378,
 'test_precision': 0.6741803278688525,
 'test_recall': 0.6741803278688525,
 'test_runtime': 1.4171,
 'test_samples_per_second': 705.672,
 'test_steps_per_second': 11.291}

### Задание 5

Хороший отзыв

In [40]:
review_10_10 = """"Twin Peaks" was a fantastic show that was quite experimental for
 its time (1990-1991) that at it's worst was better than most everything else on
  TV. I wish the show could have survived longer, but with an awful second
   season time slot - Saturday at 10pm - it had no chance. The revelation of
    Laura Palmer's killer took the wind out of the sails somewhat just nine
     shows into the second season, but the true theme of Twin Peaks was about
      the evil in the woods that took the form of the killer and the seamy
       underbelly of what appeared to be everyday small-town life. The murder of
        Laura Palmer was just the surface of that. Three somewhat pointless
        episodes followed the one that revealed Laura's killer, but then the
         show bounced back with the Windham Earle storyline. Unfortunately many
          viewers had stopped watching the show by then. Perhaps the lasting
           legacy of "Twin Peaks" is that it made series with weird subplots,
            long story arcs, and oddball characters more acceptable to the
             networks. For example, I don't think that "The X-Files" could have
              made it to the air in 1993 had it not been for "Twin Peaks"
               preceding it and succeeding. After all, after a show where there
                is a dwarf that materializes on a bed, dances, and talks
                 backwards, an alternate reality where clones are created and
                  sent out to the world, and creamed corn as a symbol of
                   suffering, the adventures of Scully and Mulder look as
                    thematically tame as Dragnet. Highly recommended."""

Плохой отзыв

In [41]:
review_1_10 = """I don't know what is it with people and old 'classic' shows that
 they feel obligated to rate so high, sure for some old folks this kinda shows
  brings back memories, but does it mean it's worthy of a high rate like 8.9 ?
   in my opinion absolutely not. i'm a fan of classic stuff , and some of you
    may say you should not be so hard on the shows like this, because well
     they're really old. that is simply not a good argument, x files was old too
      and i loved it, twin peaks is dull and not even remotely scary.

this show is consisted of a dull storyline, unfit music and stupid actors who
 have no idea of how they should act, all these elements and much more that i can't
  recall right now makes this show a cheap attempt to what i don't know, if
   their purpose was to make a horror series i have to say they failed big time,
    throughout the show i felt everything but horror, even laughing occasionally
     occasionally."""

Средний отзыв (5/10 - все бы ок, если бы не концовка)

In [42]:
review_5_10 = """Maybe that in the nineties this was one of the favorites series
 for a lot of people but compared to what they make now for top series I think
  Twin Peaks is much to much overrated. I remember David Lynch's creation had a
   lot of people talking every day after an episode but I never watched it then
    when it came out. So now I thought I give it a try and I have to say that I
     am more then disappointed about it. The first season was not bad at all,
      with enough intriguing twists to keep you interested. The actors are all
       fine as well and the filming is more then okay. But what to say from the
        ending? That is really a bad ending, he must have been out of
         inspiration. If it was not for the ending I might have score it a bit
          more but with this ending it does not deserve more then 5 stars. Glad
           I watched it so now I can talk about it when somebody brings it up
            but I will for sure never watch it a second time."""

Каждая последующая модель в этой работе показывала себя лучше, чем предыдущая. Поэтому мы возьмем последнюю модель (самую лучшую) и на ней прогоним три отзыва 

In [44]:
model_pretrained.to('cpu')
pipe = TextClassificationPipeline(model=model_pretrained, tokenizer=tokenizer)
pred = pipe([review_10_10, review_1_10, review_5_10])
pred

Disabling tokenizer parallelism, we're using DataLoader multithreading already


[{'label': 'LABEL_1', 'score': 0.5223124623298645},
 {'label': 'LABEL_0', 'score': 0.534305989742279},
 {'label': 'LABEL_1', 'score': 0.5007689595222473}]

Как мы видим, она правильно классифицировала первые два отзыва как положительный и отрицительный (LABEL_1 = положительный). Последний отзыв она классифицировала как положительный, и ее можно понять, так как там есть выражения "one of the favorites series for a lot of people", "Glad I watched it". Хотя, как по мне, это все-таки скорее негативный отзыв